![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
## PEC3
### Practica sobre la predicción del ratings de las películas

Uno de los usos más comunes de big data es predecir lo que quieren los usuarios. Esto permite que Google le muestre anuncios relevantes, Amazon le recomiende productos relevantes y Netflix le recomiende películas que le pueden gustar. Este laboratorio demostrará cómo podemos usar Apache Spark para recomendar películas a un usuario. Comenzaremos con algunas técnicas básicas y luego usaremos el método de mínimos cuadrados alternos de la biblioteca Spark ML para hacer predicciones más sofisticadas.

Para esta práctica de laboratorio, usaremos un subconjunto de datos de 100 mil calificaciones. Este conjunto de datos está premontado en Databricks y proviene del conjunto de datos de calificación de referencia estable de MovieLens. Sin embargo, el mismo código que escriba también funcionará en el conjunto de datos completo (aunque es probable que la ejecución con el conjunto de datos completo en Community Edition lleve bastante tiempo).

En este laboratorio:

Parte 0: Preliminares

Parte 1: Recomendaciones básicas

Parte 2: filtrado colaborativo

Parte 3: Predicciones para usted mismo

Como se mencionó durante el primer laboratorio de Learning Spark, piense detenidamente antes de llamar a collect () en cualquier conjunto de datos. Cuando está utilizando un conjunto de datos pequeño, llamar a collect () y luego usar Python para tener una idea de los datos localmente (en el programa del controlador) funcionará bien, pero esto no funcionará cuando esté utilizando un conjunto de datos grande que no caben en la memoria en una máquina.

In [2]:
import findspark
findspark.init()
import pyspark
import random
import os.path
sc = pyspark.SparkContext(master="local[1]",appName='Movies_ratings')

In [3]:
# set path datasets

import os
datasets_dir= '/home/gonzalezjulvez/Documentos/Projects_Github/Projects/Pyspark/data/Movies/'

movies_path = datasets_dir+'movies.csv'
ratings_path = datasets_dir+'ratings.csv'


Para la carga de los archivos vamos a acelerar aún más las cosas especificando explícitamente el esquema DataFrame. (Cuando el adaptador Spark CSV infiere el esquema de un archivo CSV, tiene que realizar una pasada adicional sobre el archivo. Eso ralentizará las cosas aquí, y no es realmente necesario).

In [4]:
from pyspark.sql.types import *

schema_movies = StructType([StructField('ID', IntegerType(),True),
                            StructField('title',StringType(),True)])

schema_ratings = StructType([StructField('userID',IntegerType(),True),
                             StructField('movieID', IntegerType(),True),
                             StructField('rating',DoubleType(),True)])

In [5]:
schema_ratings

StructType(List(StructField(userID,IntegerType,true),StructField(movieID,IntegerType,true),StructField(rating,DoubleType,true)))

## Cargar y Caché

Vamos a tener mucho acceso a estos datos. En lugar de leerlo una y otra vez, almacenaremos en caché tanto las películas DataFrame como las clasificaciones DataFrame en la memoria.


In [6]:
# Load Datasets
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
raw_df_movies = sqlContext.read.format('csv').options(header='True', delimiter=',', inferschema='False').schema(schema_movies).load(movies_path)
raw_df_ratings = sqlContext.read.format('csv').options(header='True', delimiter=',', inferschema='False').schema(schema_ratings).load(ratings_path)

# Remove useless columns. 
df_movies = raw_df_movies.drop('Genres').withColumnRenamed('movieID', 'ID')
df_ratings = raw_df_ratings.drop('Timestamp')

# Load in cache
df_movies.cache()
df_ratings.cache()

DataFrame[userID: int, movieID: int, rating: double]

In [7]:
# Count columns and show table

print('There are {0} movies with {1} ratings in our datasets'.format(df_movies.count(), df_ratings.count()))

df_movies.show(2)
df_ratings.show(2)

There are 9742 movies with 100836 ratings in our datasets
+---+----------------+
| ID|           title|
+---+----------------+
|  1|Toy Story (1995)|
|  2|  Jumanji (1995)|
+---+----------------+
only showing top 2 rows

+------+-------+------+
|userID|movieID|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
+------+-------+------+
only showing top 2 rows



## Parte 1: Recomendaciones básicas

Una forma de recomendar películas es recomendar siempre las películas con la calificación promedio más alta. En esta parte, usaremos Spark para encontrar el nombre, el número de calificaciones y la calificación promedio de las 20 películas con la calificación promedio más alta y al menos 170 reseñas. Queremos filtrar nuestras películas con calificaciones altas pero mayores o iguales a 170 reseñas porque es posible que las películas con pocas reseñas no tengan un gran atractivo para todos.

### (1a) Películas con calificaciones promedio más altas


Recuerde que ratings_df contiene tres columnas:

- El ID del usuario que calificó la película.
- ID de la película que se califica
- La nota.



In [16]:
#Create table
sqlContext.sql('DROP TABLE IF EXISTS ratings')
sqlContext.registerDataFrameAsTable(df_ratings, 'ratings')
sqlContext.sql('DROP TABLE IF EXISTS movies')
sqlContext.registerDataFrameAsTable(df_movies, 'movies')


In [17]:

movie_ids_with_avg_ratings_df = sqlContext.sql('Select movieID, COUNT(rating) as count, AVG(rating) as rating FROM ratings GROUP BY movieID')
sqlContext.sql('DROP TABLE IF EXISTS movie_ids_with_avg_ratings')
sqlContext.registerDataFrameAsTable(movie_ids_with_avg_ratings_df, 'movie_ids_with_avg_ratings')





In [23]:
sqlContext.sql('SELECT * FROM movies').show(10)

+---+--------------------+
| ID|               title|
+---+--------------------+
|  1|    Toy Story (1995)|
|  2|      Jumanji (1995)|
|  3|Grumpier Old Men ...|
|  4|Waiting to Exhale...|
|  5|Father of the Bri...|
|  6|         Heat (1995)|
|  7|      Sabrina (1995)|
|  8| Tom and Huck (1995)|
|  9| Sudden Death (1995)|
| 10|    GoldenEye (1995)|
+---+--------------------+
only showing top 10 rows



In [45]:
movie_names_with_avg_ratings_df = sqlContext.sql(" SELECT r.movieID, m.title, r.count, r.rating FROM movie_ids_with_avg_ratings r, movies m WHERE m.ID=r.movieID ORDER BY count DESC")
movie_names_with_avg_ratings_df.show(10)

+-------+--------------------+-----+-----------------+
|movieID|               title|count|           rating|
+-------+--------------------+-----+-----------------+
|    356| Forrest Gump (1994)|  329|4.164133738601824|
|    318|Shawshank Redempt...|  317|4.429022082018927|
|    296| Pulp Fiction (1994)|  307|4.197068403908795|
|    593|Silence of the La...|  279|4.161290322580645|
|   2571|  Matrix, The (1999)|  278|4.192446043165468|
|    260|Star Wars: Episod...|  251|4.231075697211155|
|    480|Jurassic Park (1993)|  238|             3.75|
|    110|   Braveheart (1995)|  237|4.031645569620253|
|    589|Terminator 2: Jud...|  224|3.970982142857143|
|    527|Schindler's List ...|  220|            4.225|
+-------+--------------------+-----+-----------------+
only showing top 10 rows



Ahora que tenemos un DataFrame de las películas con calificaciones promedio más altas, podemos usar Spark para determinar las 20 películas con calificaciones promedio más altas y al menos 170 reseñas.


In [46]:
movies_with_170_ratings = movie_names_with_avg_ratings_df[movie_names_with_avg_ratings_df['count']>=170]
movies_with_170_ratings.show(20)

+-------+--------------------+-----+------------------+
|movieID|               title|count|            rating|
+-------+--------------------+-----+------------------+
|    356| Forrest Gump (1994)|  329| 4.164133738601824|
|    318|Shawshank Redempt...|  317| 4.429022082018927|
|    296| Pulp Fiction (1994)|  307| 4.197068403908795|
|    593|Silence of the La...|  279| 4.161290322580645|
|   2571|  Matrix, The (1999)|  278| 4.192446043165468|
|    260|Star Wars: Episod...|  251| 4.231075697211155|
|    480|Jurassic Park (1993)|  238|              3.75|
|    110|   Braveheart (1995)|  237| 4.031645569620253|
|    589|Terminator 2: Jud...|  224| 3.970982142857143|
|    527|Schindler's List ...|  220|             4.225|
|   2959|   Fight Club (1999)|  218| 4.272935779816514|
|      1|    Toy Story (1995)|  215|3.9209302325581397|
|   1196|Star Wars: Episod...|  211|4.2156398104265405|
|   2858|American Beauty (...|  204| 4.056372549019608|
|     50|Usual Suspects, T...|  204| 4.237745098

## Parte 2: filtrado colaborativo

En este curso, ha aprendido sobre muchas de las transformaciones y acciones básicas que Spark nos permite aplicar a conjuntos de datos distribuidos. Spark también expone algunas funciones de nivel superior; en particular, Machine Learning usando un componente de Spark llamado MLlib. En esta parte, aprenderá a usar MLlib para hacer recomendaciones de películas personalizadas utilizando los datos de películas que hemos estado analizando